#### Setup MongoDB credentials for administrator

In [ ]:
username = "root"
password = "example"
host = "localhost"

database_name = "elections"
collection_name = "general_elections_2022"

#### Create connection string of MongoDB

In [ ]:
from utils import create_connection_string

data_lake_connection_string = create_connection_string({
    'username': username,
    'password': password,
    'host': host,
    'port': 80,
    'collection': None,
    'database': None
})

data_warehouse_connection_string = create_connection_string({
    'username': username,
    'password': password,
    'host': host,
    'port': 443,
    'collection': None,
    'database': None
})

#### Create database, collection and users if does not exists

In [ ]:
from utils import (
  create_mongodb_database_and_collection,
  create_user_owner_for_database
)

create_mongodb_database_and_collection(data_lake_connection_string, database_name, collection_name)
create_mongodb_database_and_collection(data_warehouse_connection_string, database_name, collection_name)

create_user_owner_for_database(data_lake_connection_string, database_name, "dba", "pandas")
create_user_owner_for_database(data_warehouse_connection_string, database_name, "dba", "pandas")

print(" 🛢️ Criando Usuários e Collections...")

#### Create folder and check if extraction has already happened

In [ ]:
from utils import (
  get_path_for_csv_file,
  check_if_extraction_folder_is_created
)
from utils import EXTRACTION_FOLDER_PATH

print(" 📂 Criando pasta...")

csv_file_path = get_path_for_csv_file()
check_if_extraction_folder_is_created(EXTRACTION_FOLDER_PATH)

#### Download zip file

In [ ]:
from utils import download_csv_if_not_exists

print(" ⌛ Baixando arquivo .zip e extraindo .csv...")

download_csv_if_not_exists(csv_file_path)

#### Setup credentials to load data

In [ ]:
from utils import create_connection_string

host = "0.0.0.0"
port = 80
username = 'dba'
password = 'pandas'
database = 'elections'
collection = 'general_elections_2022'

uri = create_connection_string({
  'host': host,
  'port': port,
  'username': username,
  'password': password,
  'database': database,
  'collection': collection
})

#### Read csv file

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.dataframe import DataFrame

spark_session = SparkSession.builder.appName('spark').config("spark.mongodb.output.uri", uri).config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.0").getOrCreate()

df: DataFrame = spark_session.read.options(header="true", delimiter=";", encoding="ISO-8859-1", inferSchema=True).csv(csv_file_path)

#### Load into MongoDB the csv data

In [ ]:
print(" 🖋️ Inserindo dados no MongoDB...")

df.write.format("com.mongodb.spark.sql.DefaultSource").mode("append").save()

print(" ✅ Dados escritos no MongoD!!")

#### Clear

In [ ]:
print(" 🧹 Limpando...")

spark_session.stop()